# Factorial Kriging Analysis

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os

The Grid containing the information is downloaded from the distribution.

The loaded file (called **grid **) contains 3 variables:

- **P** (phosphorus) which is the *variable of interest*
- **Cr** (chromium) is an auxiliary variable
- **Ni** (nickel) another auxiliary variable


In [ ]:
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "FKA", "Image.ascii")
grid = gl.DbGrid.createFromNF(fileNF)
ndim = 2
gl.ASpaceObject.defineDefaultSpace(gl.ESpaceType.SPACE_RN, ndim)

In [ ]:
dbfmt = gl.DbStringFormat()
dbfmt.setFlags(flag_resume=False,flag_vars=False,flag_stats=True, names="P")
grid.display(dbfmt)

Note that some pixels are not informed for variable **P**.

Statistics on auxiliary variables

In [ ]:
dbfmt.setFlags(flag_vars=False, flag_resume=True, flag_stats=True, names=["Cr", "Ni"])
grid.display(dbfmt)

Correlation between variables

In [ ]:
ax = gp.correlation(grid, "Cr", "P", figsize=[5,5])

In [ ]:
ax = gp.correlation(grid, "Ni", "P", figsize=[5,5])

In [ ]:
ax = gp.correlation(grid, "Ni", "Cr", figsize=[5,5])

Using inverse square distance for completing the variable **P**

In [ ]:
grid.setLocator("P", gl.ELoc.Z)
err = gl.db_grid_fill(grid)

We concentrate on the variable of interest **P** completed (**Fill.P**) in the next operations

In [ ]:
ax = gp.grid(grid,"Fill.P",figsize=[8,8],aspect=1)

Variogram Calculation along Grid main axes

In [ ]:
varioparam = gl.VarioParam.createMultipleFromGrid(npas=100)
varioP = gl.Vario(varioparam, grid)
err = varioP.compute()
modelP = gl.Model()
err = modelP.fit(varioP, types=[gl.ECov.NUGGET, gl.ECov.SPHERICAL, gl.ECov.POWER],
                 optvar=gl.Option_VarioFit(True, False))

In [ ]:
ax = gp.varmod(varioP, modelP)

We must define the **Neighborhood**

In [ ]:
neigh = gl.NeighImage([10,10])

The **image** neighborhood is based on $(2*10+1)^2=441$ pixels (centered on the target pixel).

During the estimation, only the contribution of second and third basic structures are kept (Nugget Effect is filtered out): ** Factorial Kriging Analysis**.

In [ ]:
modelP.setCovaFiltered(0, True)
means = gl.dbStatisticsMono(grid,["Fill.P"],[gl.EStatOption.MEAN])
modelP.setMeans(means)
modelP

In [ ]:
err  = gl.krimage(grid,modelP,neigh,namconv=gl.NamingConvention("Mono"))

In [ ]:
ax = gp.plot(grid,"Mono*.P", figsize=[8,8],aspect=1,title="P denoised (monovariate)")

Correlation for **P** variable between Initial image (completed) and its Filtered version (monovariate FKA)

In [ ]:
grid

In [ ]:
ax = gp.correlation(grid, "Fill.P", "Mono.Fill.P", figsize=[5,5],
                    xlabel="P Filled",ylabel="P Filtered (Mono)")

## Multivariate approach

In [ ]:
grid.setLocators(["Fill.P", "Cr", "Ni"], gl.ELoc.Z)

varioM = gl.Vario(varioparam, grid)
err = varioM.compute()
modelM = gl.Model()
err = modelM.fit(varioM, types=[gl.ECov.NUGGET, gl.ECov.SPHERICAL, gl.ECov.POWER],
                 optvar=gl.Option_VarioFit(True, False))

In [ ]:
ax = gp.varmod(varioM, modelM, figsize=[4,4])

Printing the contents of the fitted Multivariate Mpdel

In [ ]:
modelM.setCovaFiltered(0, True)
means = gl.dbStatisticsMono(grid,["Fill.P", "Cr", "Ni"],[gl.EStatOption.MEAN])
modelM.setMeans(means)
modelM

Multivariable Factorial Kriging Analysis

In [ ]:
err  = gl.krimage(grid,modelM,neigh,namconv=gl.NamingConvention("Multi"))

Note that, using the same **neigh** as in monovariate, the dimension of the Kriging System is now $3 * 441 = 1323$

In [ ]:
ax = gp.plot(grid,"Multi*.P", figsize=[8,8],aspect=1,title="P denoised (multivariate)")

Correlation for **P** variable between Initial image and its Filtered version (multivariate FKA)

In [ ]:
ax = gp.correlation(grid, "Fill.P", "Multi.Fill.P", figsize=[5,5],
                    xlabel="P Filled",ylabel="P Filtered (Multi)")

Correlation for **P** filtered variable between he Monovariate and the Multivariate case

In [ ]:
ax = gp.correlation(grid, "Mono.Fill.P", "Multi.Fill.P", figsize=[5,5],
                    xlabel="P Filtered (Mono)",ylabel="P Filtered (Multi)")